In [1]:
# -*- coding: utf-8 -*-
"""
19:28:32, Thur, 30 March,  2023

By Gr

Edition time: 2023:4.5
"""

import numpy as np
import numpy.random as rnd
import torch





# setting up GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# load the data from given dataset


print(torch.__version__)


print(torch.cuda.device_count())

2.0.0+cu118
1


In [2]:
#Define funtion that Get data and create train and test dataset
from torch.utils.data import DataLoader,random_split
class myDataset():
    def __init__(self,data_dir,label_dir):
    #data structure is 100x256xN
    #Label structure is N(0~7)
        data = np.load(data_dir)
        label = np.load(label_dir)

    # now convert integer to multinomial rerepsentation for classfication
    # one-hot code
    
        label_multi = np.zeros((label.shape[0],7))
        for i in range(label.shape[0]):
            tmp = round(label[i])-1
            label_multi[i,tmp]=1

        self.data = torch.tensor(data).float()
        self.label = torch.tensor(label_multi).float()
        
        del label_multi,data
    
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, index):
        return self.data[:,:,index],self.label[index,:]

def get_dataloader(data_dir, label_dir, batch_size, n_workers, train_len):
#In this lab, every feature has same length, so don't need padding
    dataset = myDataset(data_dir,label_dir)
    #for example, train_len = 0.8 means 80% of data will be assigned to training set
    trainlen = int(train_len * len(dataset))
    lengths = [trainlen, len(dataset)-trainlen]
    #split trainset and validset
    trainset, validset = random_split(dataset,lengths)

    train_loader = DataLoader(
        trainset,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=n_workers,
        pin_memory=True,
  )
    valid_loader = DataLoader(
        validset,
        batch_size=batch_size,
        num_workers=n_workers,
        drop_last=True,
        pin_memory=True,
  )
    return train_loader, valid_loader

In [3]:
#Define Transformer Model
import torch
import torch.nn as nn

class TransformerClassifier(nn.Module):
    def __init__(self, d_model, n_class, dropout):
        super().__init__()
    #d_model = 256 channels, n_class = 7, 
       

        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model = d_model, dim_feedforward = 256, nhead = 8, 
            dropout = dropout
        )
        self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)

        #Decoder Part need a Memory as an input
        '''
        self.decoder_layer = nn.TransformerDecoderLayer(
            d_model = d_model, nhead = 8
        )
        '''
        
        self.pred_layer = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
            nn.Linear(d_model, n_class),
        )
    
    
    
    def forward(self, mels):
        """
    args:
      mels: (batch size, length, 256)
    return:
      out: (batch size, n_class)
        """
        # The encoder layer expect features in the shape of (length, batch size, d_model).
        out = self.encoder(mels)

        # out: (batch size, length, d_model)
        #out = out.transpose(0, 1)
        # mean pooling
        stats = out.mean(dim=1)

        # out: (batch, n_spks)
        out = self.pred_layer(stats)
        return out

In [4]:
#Define LSTM Model
import torch
import torch.nn as nn
class rnn_gru(nn.Module):
    def __init__(self,num_classes,input_size,hidden_size,num_layers,dropout_val):
        super(rnn_gru,self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size       
        
        self.rnn1=nn.LSTM(input_size=input_size,hidden_size=hidden_size,
                          num_layers=num_layers,batch_first=True,dropout=dropout_val,
                          bidirectional=True)
        self.rnn2=nn.GRU(input_size=round(hidden_size*2),hidden_size=round(hidden_size/2),
                          num_layers=num_layers,batch_first=True, bidirectional=False)
        
        self.mlp_input = round(hidden_size/2)
        self.linear1 = nn.Linear(self.mlp_input,num_classes)
    
    def forward(self,x):        
        output1, (hn1,cn1) = self.rnn1(x) 
        output2, (hn2) = self.rnn2(output1)
        hn2 = torch.squeeze(hn2)        
        out = self.linear1(hn2)        
        return out

In [5]:
#Define train function
from tqdm import tqdm
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    total_correct = 0

    train_iter = iter(train_loader)
    
    #pbar = tqdm(total = 199, ncols = 100 )
    #Looping all the train_loader
    '''
    #There is also an alternative for you to write

    for mel, label in range emunerate(train_loader):
        optimzer.zero_grad()
    '''
    
    for i in range(len(train_iter)):
        mel, label = next(train_iter)
        mel, label = mel.to(device), label.to(device)
        optimizer.zero_grad()
        output = model(mel)
        loss = criterion(output,label)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * mel.shape[0]
        total_correct += (output.argmax(dim=1) == label.argmax(dim=1)).sum().item()

        '''
        pbar.update()
        pbar.set_postfix(
            loss = f"{loss.item() * mel.shape[0]/len(mel):.2f}",
            accuracy = f"{(output.argmax(dim=1) == label.argmax(dim=1)).sum().item()/len(mel):.2f}",
        )
        '''    
            
        #if (i%10 == 0):
            #print(f'Iteration = {i},Accuracy = {(output.argmax(dim=1) == label.argmax(dim=1)).sum().item()/ len(label)}')
    #pbar.close()
    print(f'Training_loss = {total_loss / len(train_loader.dataset):.4f}, Training_Accuracy = {total_correct / len(train_loader.dataset) * 100 :.2f}%',end = ' ')
    return total_loss / len(train_loader.dataset), total_correct / len(train_loader.dataset)

In [6]:
#Define valid function, return loss, accuracy, best_loss
def valid(model,valid_loader,criterion,save_path):
    model.eval()
    total_loss = 0
    total_correct = 0
    #Save best model    
    best_loss = 2


    valid_iter = iter(valid_loader)
    for i in range(len(valid_iter)):
        with torch.no_grad():
            mel, label = next(valid_iter)
            mel, label = mel.to(device), label.to(device)
            output = model(mel)
            loss = criterion(output,label)

            if (loss < best_loss):
                best_loss = loss
                torch.save(model.state_dict(), save_path)
                
        total_loss += loss.item() * mel.shape[0]
        total_correct += (output.argmax(dim=1) == label.argmax(dim=1)).sum().item()
    model.train()

    print(f'Valid_loss = {total_loss / len(valid_loader.dataset):.4f}, Valid_Accuracy = {total_correct / len(valid_loader.dataset) * 100 :.2f}%')
    return total_loss / len(valid_loader.dataset), total_correct / len(valid_loader.dataset), best_loss

In [7]:
#Load data, using dataloader spilt into training set and test set(validation set)
# mel, label = batch (N, 100, 256)
data_dir = 'data/condn_data_new.npy'  #TODO condn_data_new.npy path
label_dir = 'data/Y.npy'              #TODO Y.npy path
batch_size = 200
n_workers = 0
train_len = 0.8
train_loader, valid_loader = get_dataloader(data_dir, label_dir, batch_size, n_workers, train_len)

In [8]:
#Transformer model
data_dir = 'data/condn_data_new.npy'  #TODO condn_data_new.npy path
label_dir = 'data/Y.npy'              #TODO Y.npy path
weight_path = 'model/save.pt'
batch_size = 200
n_workers = 0
train_len = 0.8
d_model = 256
n_class = 7
dropout = 0.7
lr = 1e-4
num_epochs = 100
in_channel = 1
model = TransformerClassifier(d_model, n_class, dropout)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = torch.nn.CrossEntropyLoss()



In [9]:
#LTSM model
'''
data_dir = 'data/condn_data_new.npy'  #TODO condn_data_new.npy path
label_dir = 'data/Y.npy'              #TODO Y.npy path
# lstm parameters
input_size=256
hidden_size=150
num_layers=1
sequence_length = 100
num_classes=7
dropout_val=0.3
n_workers = 0

#basic parametres
data_dir = 'data/condn_data_new.npy'  #TODO condn_data_new.npy path
label_dir = 'data/Y.npy'              #TODO Y.npy path
batch_size = 200
n_workers = 0
train_len = 0.8
d_model = 256
n_class = 7
dropout = 0.5
lr = 1e-3
num_epochs = 100

model = rnn_gru(num_classes,input_size,hidden_size,num_layers,dropout_val)
model = model.to(device) #push to GPU

num_epochs=100
batch_size = 256
gradient_clipping = 10.0
learning_rate=2e-4
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
criterion = nn.CrossEntropyLoss()
'''


"\ndata_dir = 'data/condn_data_new.npy'  #TODO condn_data_new.npy path\nlabel_dir = 'data/Y.npy'              #TODO Y.npy path\n# lstm parameters\ninput_size=256\nhidden_size=150\nnum_layers=1\nsequence_length = 100\nnum_classes=7\ndropout_val=0.3\nn_workers = 0\n\n#basic parametres\ndata_dir = 'data/condn_data_new.npy'  #TODO condn_data_new.npy path\nlabel_dir = 'data/Y.npy'              #TODO Y.npy path\nbatch_size = 200\nn_workers = 0\ntrain_len = 0.8\nd_model = 256\nn_class = 7\ndropout = 0.5\nlr = 1e-3\nnum_epochs = 100\n\nmodel = rnn_gru(num_classes,input_size,hidden_size,num_layers,dropout_val)\nmodel = model.to(device) #push to GPU\n\nnum_epochs=100\nbatch_size = 256\ngradient_clipping = 10.0\nlearning_rate=2e-4\noptimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)\ncriterion = nn.CrossEntropyLoss()\n"

In [10]:
#Training
num_epochs = 100
train_loss = []
train_accuracy = []
valid_loss = []
valid_accuracy = []
for epoch in range(num_epochs):
    loss,accuracy = train(model,train_loader,optimizer,criterion,device)
    train_loss.append(loss), train_accuracy.append(accuracy)
    loss,accuracy,best_loss = valid(model,valid_loader,criterion,weight_path)
    valid_loss.append(loss), valid_accuracy.append(accuracy)


Training_loss = 1.7831, Training_Accuracy = 26.37% Valid_loss = 1.5370, Valid_Accuracy = 36.35%
Training_loss = 1.5073, Training_Accuracy = 39.47% Valid_loss = 1.4156, Valid_Accuracy = 41.45%
Training_loss = 1.3916, Training_Accuracy = 44.75% Valid_loss = 1.3408, Valid_Accuracy = 45.46%
Training_loss = 1.3090, Training_Accuracy = 49.06% Valid_loss = 1.2257, Valid_Accuracy = 50.83%
Training_loss = 1.2376, Training_Accuracy = 52.35% Valid_loss = 1.2114, Valid_Accuracy = 53.03%
Training_loss = 1.1820, Training_Accuracy = 54.95% Valid_loss = 1.1625, Valid_Accuracy = 54.66%
Training_loss = 1.1441, Training_Accuracy = 56.21% Valid_loss = 1.0674, Valid_Accuracy = 58.75%
Training_loss = 1.1089, Training_Accuracy = 58.15% Valid_loss = 1.0676, Valid_Accuracy = 59.04%
Training_loss = 1.0702, Training_Accuracy = 59.83% Valid_loss = 1.0523, Valid_Accuracy = 59.94%
Training_loss = 1.0537, Training_Accuracy = 60.50% Valid_loss = 1.0059, Valid_Accuracy = 61.67%
Training_loss = 1.0217, Training_Accurac

KeyboardInterrupt: 

In [ ]:
#Load model dict
model = TransformerClassifier(d_model,n_class,dropout)
model.load_state_dict(torch.load(weight_path))
model.eval()

In [ ]:
#valid(model,valid_loader,criterion)
import matplotlib.pyplot as plt
plt.figure()
plt.plot(train_loss)
plt.plot(valid_loss)


In [ ]:
plt.figure()
plt.plot(train_accuracy)
plt.plot(valid_accuracy)

In [ ]:
#Save to Excel
import numpy as np
import pandas as pd
def  writeto_excel(train_loss, train_accuracy,valid_loss,valid_accuracy):
    '''
    ##Loss: List, consist of num_epochs results
    ##Accuracy: List, consist of num_epochs results
    To be noted, results would be consist model's result from scratch
    '''
    #Confer list to array for pandas process
    data = []
    for i in range(len(train_loss)): 
        data.append([train_loss[i], train_accuracy[i],
                     valid_loss[i], valid_accuracy[i]])
    data = np.array(data)
    data = pd.DataFrame(data)
    writer = pd.ExcelWriter('Result.xlsx')
    data.to_excel(writer, 'page_1', float_format='%.2f')
    writer.save()

    print('The Loading Process has been done')
    writer.close()
writeto_excel(train_loss,train_accuracy,valid_loss,valid_accuracy)
    